### Données de 18/02/2019 00:00 jusqu'à 18/02/2019 16:22 de toutes les stations

In [1]:
#Charger les biblio
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import numpy as np
import string 
from subprocess import check_output
import seaborn as sns
from pylab import *
%matplotlib inline
sns.set()

In [2]:
# Lire la table avec 'header'
df = pd.read_csv('Hmvl2.csv', sep=';', engine='python')

In [3]:
# Remplacer les NaN par des zeros dans trame_hmvl
df.trame_hmvl.fillna('0', inplace=True)

In [4]:
# Reindexer la dataframe
df.index = range(len(df))

In [5]:
#changeaons les noms de colonnes
df.columns = ['code', 'id','equipement_id', 'date','trame']

In [6]:
df.head()

,code,id,equipement_id,date,trame
0,MBS,7357,7357,2019-02-18 00:00:00.146,0543110903705431109037256930750412569307504125...
1,MPA,7356,7356,2019-02-18 00:00:00.146,@
2,MPH,7355,7355,2019-02-18 00:00:00.146,058520920440585209204405852092044@
3,SFB-TPCCEN_RDTSCB001,7850,7850,2019-02-18 00:00:01.374,@
4,SMA-INTBAU_RDTSCB001,7859,7859,2019-02-18 00:00:01.374,@


In [9]:
# On enlève les @
df["trame"] = df["trame"].str.replace("@", "")

In [10]:
# On ajoute une colonne Count pour compter chacune
df['Count'] = df['trame'].str.len()

In [11]:
df.head()

,code,id,equipement_id,date,trame,Count
0,MBS,7357,7357,2019-02-18 00:00:00.146,0543110903705431109037256930750412569307504125...,55
1,MPA,7356,7356,2019-02-18 00:00:00.146,,0
2,MPH,7355,7355,2019-02-18 00:00:00.146,058520920440585209204405852092044,33
3,SFB-TPCCEN_RDTSCB001,7850,7850,2019-02-18 00:00:01.374,,0
4,SMA-INTBAU_RDTSCB001,7859,7859,2019-02-18 00:00:01.374,,0


In [12]:
df.Count.max()/11

46.0

In [13]:
# Supprimons les lignes differentes de zero
df = df[df.Count != 0]

In [14]:
# Supprimons les lignes differentes de 1
df = df[df.Count != 1]

In [15]:
df.groupby("Count").count().head()

,code,id,equipement_id,date,trame
Count,,,,,
5,1,1,1,1,1
11,149327,149327,149327,149327,149327
12,245,245,245,245,245
16,1,1,1,1,1
22,106444,106444,106444,106444,106444


In [16]:
dff = df.groupby("Count").count()

In [17]:
# La somme cumulé
dff['CUMS'] = dff['id'].cumsum(axis = 0)

In [18]:
dff[60:75]

,code,id,equipement_id,date,trame,CUMS
Count,,,,,,
107,1,1,1,1,1,827858
108,1,1,1,1,1,827859
110,33234,33234,33234,33234,33234,861093
111,149,149,149,149,149,861242
112,130,130,130,130,130,861372
113,89,89,89,89,89,861461
114,43,43,43,43,43,861504
115,15,15,15,15,15,861519
116,1,1,1,1,1,861520


Le nombre totale de caractère qui représente $100\%$ est de $929852$.

$884431$ représente $95.11\%$ de données qui correspond à la valeur $121$ au champs 'Count'.

On considère alors, pour ces données, le nombre maximal de caractère est de $11$.

Donc il faut créer 11 nouveaux champs qui contient les 11 caractères pour chaque véhicules.

In [19]:
# Supprimons la colonne
df = df.drop(['Count'], axis=1)

In [20]:
liste = ["x0","x1","x2","x3","x4","x5","x6","x7","x8","x9",'x10']

In [21]:
# Ajouter les colonnes, en séparant la colonne 'trame' 
for i in range(len(liste)):
    df[liste[i]] = df["trame"].str.slice(11*i,11*(i+1))

In [22]:
df.head(3)

,code,id,equipement_id,date,trame,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,MBS,7357,7357,2019-02-18 00:00:00.146,0543110903705431109037256930750412569307504125...,05431109037,05431109037,25693075041,25693075041,25896106044,,,,,,
2,MPH,7355,7355,2019-02-18 00:00:00.146,058520920440585209204405852092044,05852092044,05852092044,05852092044,,,,,,,,
6,M1b,6383,6383,2019-02-18 00:00:02.069,15736085041,15736085041,,,,,,,,,,


In [23]:
# Supprimer les colonnes xj
df0 = df.drop(['x3','x1','x2','x4','x5','x6','x7','x8', 'x9', 'x10', 'trame'], axis=1)
# df1
df1 = df.drop(['x3','x0','x2','x4','x5','x6','x7','x8', 'x9', 'x10',  'trame'], axis=1)
# Supprimer les colonnes xj
df2 = df.drop(['x3','x1','x0','x4','x5','x6','x7','x8', 'x9', 'x10', 'trame'], axis=1)
df3 = df.drop(['x0','x1','x2','x4','x5','x6','x7','x8', 'x9', 'x10', 'trame'], axis=1)
# Supprimer les colonnes xj
df4 = df.drop(['x0','x1','x2','x3','x5','x6','x7','x8', 'x9', 'x10', 'trame'], axis=1)
# Supprimer les colonnes xj
df5 = df.drop(['x0','x1','x2','x4','x3','x6','x7','x8', 'x9', 'x10', 'trame'], axis=1)
# Supprimer les colonnes xj
df6 = df.drop(['x0','x1','x2','x4','x5','x3','x7','x8', 'x9', 'x10', 'trame'], axis=1)
# Supprimer les colonnes xj
df7 = df.drop(['x0','x1','x2','x4','x5','x6','x3','x8', 'x9', 'x10', 'trame'], axis=1)
# Supprimer les colonnes xj
df8 = df.drop(['x0','x1','x2','x4','x5','x6','x3','x7', 'x9', 'x10', 'trame'], axis=1)
df9 = df.drop(['x0','x1','x2','x4','x5','x6','x3','x7', 'x8', 'x10',  'trame'], axis=1)

df10 = df.drop(['x0','x1','x2','x4','x5','x6','x3','x7', 'x9', 'x8','trame'], axis=1)


In [24]:
liste = ["x0","x1","x2","x3","x4","x5","x6","x7","x8","x9",'x10']
liste2 = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]

    # Séparer la colonne xi en 4 colonnes  
for i, f in zip(range( len(liste)), liste2):
    f["voie"] = f[liste[i]].str.slice(0,1)
    f["horodatage"] = f[liste[i]].str.slice(1,5)
    f["vitesse"] = f[liste[i]].str.slice(5,8)
    f["longueur"] = f[liste[i]].str.slice(8,11)

In [25]:
df0.head()

,code,id,equipement_id,date,x0,voie,horodatage,vitesse,longueur
0,MBS,7357,7357,2019-02-18 00:00:00.146,05431109037,0,5431,109,037
2,MPH,7355,7355,2019-02-18 00:00:00.146,05852092044,0,5852,092,044
6,M1b,6383,6383,2019-02-18 00:00:02.069,15736085041,1,5736,085,041
8,M1d,6385,6385,2019-02-18 00:00:02.069,15004114040,1,5004,114,040
9,M1e,6386,6386,2019-02-18 00:00:02.069,12780093040,1,2780,093,040


In [26]:
# rename columns
for i,f in zip(range( len(liste)), liste2):
    
    f.rename(columns={liste[i]:'x'}, inplace=True)

In [27]:
df2.head()

,code,id,equipement_id,date,x,voie,horodatage,vitesse,longueur
0,MBS,7357,7357,2019-02-18 00:00:00.146,25693075041,2,5693,075,041
2,MPH,7355,7355,2019-02-18 00:00:00.146,05852092044,0,5852,092,044
6,M1b,6383,6383,2019-02-18 00:00:02.069,,,,,
8,M1d,6385,6385,2019-02-18 00:00:02.069,,,,,
9,M1e,6386,6386,2019-02-18 00:00:02.069,,,,,


In [29]:
# Concatener les data
result = pd.concat(liste2)

In [30]:
# Reindexer la dataframe
result.index = range(len(result))

In [31]:
# Supprimons la colonne
df = result.drop(['x'], axis=1)

In [32]:
c = ['horodatage', 'vitesse','longueur']
for i in c:
    df[i] = pd.to_numeric(df[i] ,errors='coerce')

In [33]:
# Eliminer les lignes vides
df = df.dropna(axis = 0, how ='any', thresh = None, subset = None, inplace = False)

In [34]:
len(df)

4463421

In [32]:
#Exporter la table
df.to_csv('df_5sc_to.csv', index = False, encoding = 'utf-8')

 Voici le lien vers la table de données de 18/02/2019 00:00 jusqu'à 18/02/2019 16:22 de toutes les satations :
 
 https://github.com/Oumayma13/Data_5sc
 
 Sous le nom de 'df_5sc_to'.

In [33]:
# On ajoute une colonne Count pour compter le nombre de caractère de la station.
df['Count'] = df['code'].str.len()

In [237]:
df.head()

,code,id,equipement_id,date,voie,horodatage,vitesse,longueur,Count
0,MBS,7357,7357,2019-02-18 00:00:00.146,0,5431.0,109.0,037,3
1,MPH,7355,7355,2019-02-18 00:00:00.146,0,5852.0,92.0,044,3
2,M1b,6383,6383,2019-02-18 00:00:02.069,1,5736.0,85.0,041,3
3,M1d,6385,6385,2019-02-18 00:00:02.069,1,5004.0,114.0,040,3
4,M1e,6386,6386,2019-02-18 00:00:02.069,1,2780.0,93.0,040,3


In [46]:
# Supprimons la colonne
df = df.drop(['Count'], axis=1)

In [46]:
def prc(part, whole):
    return 100 * float(part)/float(whole)

In [35]:
df1 = df[(df['Count'] == 3)]
prc(len(df1),len(df))

89.56027934557389

In [36]:
df2 = df[(df['Count'] == 20)]
prc(len(df1),len(df))

89.56027934557389

In [37]:
dff = df.groupby("code").count()

In [39]:
dff.head()

,id,equipement_id,date,voie,horodatage,vitesse,longueur,Count
code,,,,,,,,
M1A,22874,22874,22874,22874,22874,22874,22874,22874
M1B,22937,22937,22937,22937,22937,22937,22937,22937
M1C,17696,17696,17696,17696,17696,17696,17696,17696
M1D,36337,36337,36337,36337,36337,36337,36337,36337
M1E,23488,23488,23488,23488,23488,23488,23488,23488


In [40]:
dff['prc'] = 100 * dff["id"]/len(df)
dff.head()

,id,equipement_id,date,voie,horodatage,vitesse,longueur,Count,prc
code,,,,,,,,,
M1A,22874,22874,22874,22874,22874,22874,22874,22874,0.511884
M1B,22937,22937,22937,22937,22937,22937,22937,22937,0.513294
M1C,17696,17696,17696,17696,17696,17696,17696,17696,0.396009
M1D,36337,36337,36337,36337,36337,36337,36337,36337,0.813165
M1E,23488,23488,23488,23488,23488,23488,23488,23488,0.525625


In [41]:
len(dff)

139

Cette table contient les données de $139$ stations, le champs prc représente le pourcentage de données de chaque stations par rapport au données totale.

In [262]:
# Supprimons toutes les colonnes de la table 'dff'
# Supprimons la colonne
dff = dff.drop(['longueur','vitesse','horodatage','voie','date', 'id' , 'equipement_id'], axis=1)

In [264]:
#Exporter la table
dff.to_csv('df_STATIONS.csv', index = False, encoding = 'utf-8')

In [67]:
dff0 = df.groupby("voie").mean()

In [68]:
# Supprimons la colonne
dff0 = dff0.drop([ 'id', 'horodatage' , 'equipement_id'], axis=1)

In [69]:
dff0

,vitesse,longueur
voie,,
0,82.271311,51.014575
1,90.436126,44.617491
2,92.187190,45.348501
3,96.927921,54.805897
4,91.692167,48.983646
5,87.282360,51.809884
6,121.420416,86.699003
7,115.148969,77.087473
8,465.935662,470.429534


In [51]:
#Exporter la table
dff0.to_csv('df_voie.csv', index = False, encoding = 'utf-8')

In [70]:
dff0.T

voie,0,1,2,3,4,5,6,7,8,9
vitesse,82.271311,90.436126,92.187190,96.927921,91.692167,87.282360,121.420416,115.148969,465.935662,449.443056
longueur,51.014575,44.617491,45.348501,54.805897,48.983646,51.809884,86.699003,77.087473,470.429534,468.994444


Voyons la longueur du champs x dans la table 'result'.

In [38]:
# On ajoute une colonne Count pour compter chacune
result['len'] = result['x'].str.len()

In [44]:
# Eliminer les lignes = 0
result = result[result.len != 0]

In [45]:
result

,code,id,equipement_id,date,x,voie,horodatage,vitesse,longueur,len
0,MBS,7357,7357,2019-02-18 00:00:00.146,05431109037,0,5431,109,037,11
1,MPH,7355,7355,2019-02-18 00:00:00.146,05852092044,0,5852,092,044,11
2,M1b,6383,6383,2019-02-18 00:00:02.069,15736085041,1,5736,085,041,11
3,M1d,6385,6385,2019-02-18 00:00:02.069,15004114040,1,5004,114,040,11
4,M1e,6386,6386,2019-02-18 00:00:02.069,12780093040,1,2780,093,040,11
5,M3D,6320,6320,2019-02-18 00:00:02.069,02210086111,0,2210,086,111,11
6,M3i,6340,6340,2019-02-18 00:00:02.069,22784088057,2,2784,088,057,11
7,M3j,6341,6341,2019-02-18 00:00:02.069,45272083045,4,5272,083,045,11
8,M3p,6347,6347,2019-02-18 00:00:02.069,33775082045,3,3775,082,045,11
9,M3q,6348,6348,2019-02-18 00:00:02.069,42638085038,4,2638,085,038,11


In [47]:
r1 = result[(result['len'] != 11)]
prc(len(r1),len(result))

0.07811651438725459

Donc il existe 0.078% d'observations dont la longueur de 'x' est différentes que 11.

In [60]:
# Supprimons ces observations.
df = result[result.len == 11]

In [61]:
# Vérif
d = df[(df['len'] == 11)]
prc(len(d),len(df))

100.0

### Rassembler les données par tranche de 6mn


In [62]:
df['vitesse'] = pd.to_numeric(df['vitesse'] ,errors='coerce')

C:\Users\oumayma.doughmi\ANACONDA\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
df = df.drop([ 'len', 'x' , 'equipement_id'], axis=1)
df.head()

,code,id,date,voie,horodatage,vitesse,longueur
0,MBS,7357,2019-02-18 00:00:00.146,0,5431,109.0,037
1,MPH,7355,2019-02-18 00:00:00.146,0,5852,92.0,044
2,M1b,6383,2019-02-18 00:00:02.069,1,5736,85.0,041
3,M1d,6385,2019-02-18 00:00:02.069,1,5004,114.0,040
4,M1e,6386,2019-02-18 00:00:02.069,1,2780,93.0,040


In [53]:
d = df.groupby("code").mean()

In [54]:
# library datetime
from datetime import datetime
from pandas import datetime

In [57]:
# convertir to datetime
df['date'] = pd.to_datetime(df['date'])
# Set the datetime column as the index
df.index = df['date']

In [58]:
# rassembler par une minute.
df_1T = df.resample('1T').mean()
# 77 obs

In [59]:
df_1T.head()

,id,vitesse
date,,
2019-02-18 00:00:00,6594.427558,97.619209
2019-02-18 00:01:00,6628.789268,98.526608
2019-02-18 00:02:00,6673.777895,96.069429
2019-02-18 00:03:00,6656.935385,94.709071
2019-02-18 00:04:00,6605.225564,94.561983


In [ ]:
df0 = df.groupby("code").mean()

In [ ]:
# Champs débit
df['Débit'] = df0['count']